In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings; 
import easyquotation
import json
import requests

In [2]:
# from yahoo_historical import Fetcher
# data = Fetcher("01299.HK", [2007,1,1], [2017,1,1])
# display(data.get_dividends())
# 財報:
# http://www.aastocks.com/tc/stocks/analysis/company-fundamental/earnings-summary?symbol=00700
# 港股列表:
# https://stock360.hkej.com/stockList/all/20200423?&p=1
# !/usr/bin/python
# coding:utf-8

import requests as rq
from bs4 import BeautifulSoup
from lxml.cssselect import CSSSelector
import io
import time

def sleeptime(hour,min,sec):
    return hour*3600 + min*60 + sec;

tStart = time.time()#計時開始

hkStockListBaseUrl = "https://stock360.hkej.com/stockList/all/20200423?&p={}"
urlPage = 0;
currentPage = 0;
stockCodeList = []
stockNameList = []

stockListDf = pd.DataFrame() 

while True:
    urlPage = urlPage + 1;
    url = hkStockListBaseUrl.format(str(urlPage))
    response = rq.get(url)
    soup = BeautifulSoup(response.text, "lxml")

    pageEle = soup.select("div[class=pagingWrap] > span[class=on]")
    currentPage = int(pageEle[0].getText())
    if (int(currentPage) != int(urlPage) ) or urlPage > 20:
        break
        
    table = pd.read_html(response.text, attrs={'class': 'tbl_sl'} , converters={'股票代號': str} )
    tempDf = table[0].iloc[1:]
    tempDf = tempDf.iloc[0:,0:3]
    tempDf.columns = ['code', 'name', 'price']
    stockListDf = stockListDf.append(tempDf, ignore_index = True)

        
    print("Success get No. %d page , url: %s" % (urlPage , url) )
    time.sleep(sleeptime(0,0,0.5))

print ("Retrived %d stock " % (len(stockListDf)))#會自動做近位
stockCodeList = stockListDf['code'].tolist()
stockNameList = stockListDf['name'].tolist()
tEnd = time.time()#計時結束
print ("It cost %f sec" % (tEnd - tStart))#會自動做近位
display(stockListDf)

Success get No. 1 page , url: https://stock360.hkej.com/stockList/all/20200423?&p=1
Success get No. 2 page , url: https://stock360.hkej.com/stockList/all/20200423?&p=2
Success get No. 3 page , url: https://stock360.hkej.com/stockList/all/20200423?&p=3
Success get No. 4 page , url: https://stock360.hkej.com/stockList/all/20200423?&p=4
Success get No. 5 page , url: https://stock360.hkej.com/stockList/all/20200423?&p=5
Success get No. 6 page , url: https://stock360.hkej.com/stockList/all/20200423?&p=6
Success get No. 7 page , url: https://stock360.hkej.com/stockList/all/20200423?&p=7
Success get No. 8 page , url: https://stock360.hkej.com/stockList/all/20200423?&p=8
Success get No. 9 page , url: https://stock360.hkej.com/stockList/all/20200423?&p=9
Success get No. 10 page , url: https://stock360.hkej.com/stockList/all/20200423?&p=10
Retrived 2713 stock 
It cost 14.334836 sec


,code,name,price
0,00001,長江和記實業s,57.00
1,00002,中電控股s,81.50
2,00003,香港中華煤氣s,13.76
3,00004,九龍倉集團s,13.98
4,00005,匯豐控股s,38.80
...,...,...,...
2708,83186,中金金瑞中網－Ｒs,59.75
2709,83188,華夏滬深三百－Ｒs,39.20
2710,83197,華夏Ａ國際通－Ｒs,0.00
2711,83199,南方政銀國債－Ｒs,106.60


In [31]:
# http://www.aastocks.com/tc/stocks/analysis/company-fundamental/earnings-summary?symbol=00700
import html5lib
baseFinanceUrl = "http://www.aastocks.com/tc/stocks/analysis/company-fundamental/earnings-summary?symbol={}"

tStart = time.time()#計時開始

allStockDf = pd.DataFrame() 
print("start")
for index, code in enumerate(stockCodeList):
# for code in ['00700']:
    url = baseFinanceUrl.format(str(code));

    r=requests.get(url,timeout=20)
    financeTable = pd.read_html(r.content, attrs={'class': 'mar15T'})
    if code.isspace() or len(financeTable[0]) < 5:
        continue
    
    financeFrame = financeTable[0].T
    financeFrame = financeFrame[:-1]
    
   # 把第一行成為HEADER
    newHeader = financeFrame.iloc[0] #grab the first row for the header
    financeFrame = financeFrame[1:] #take the data less the header row
    financeFrame.columns = newHeader #set the header row as the df header
    
    financeFrame['code'] = code
    financeFrame['name'] = stockNameList[index]
    
    # 把code和name放到前排column:
    cols = financeFrame.columns.tolist()
    cols = cols[-2:] + cols[:-2]
    financeFrame = financeFrame[cols]
    
    allStockDf = allStockDf.append(financeFrame, ignore_index = True) 

    print("Getting %s %s " % (code , stockNameList[index]) )
    time.sleep(sleeptime(0,0,0.3))
    
tEnd = time.time()#計時結束
print ("It cost %f sec" % (tEnd - tStart))#會自動做近位

allStockDf.to_csv('./all.csv', encoding="utf_8_sig" , index=False)

start
Getting 00001 長江和記實業s 
Getting 00002 中電控股s 
Getting 00003 香港中華煤氣s 
Getting 00004 九龍倉集團s 
Getting 00005 匯豐控股s 
Getting 00006 電能實業s 
Getting 00007 香港金融集團 
Getting 00008 電訊盈科s 
Getting 00009 金奧國際 
Getting 00010 恒隆集團s 
Getting 00011 恒生銀行s 
Getting 00012 恒基地產s 
Getting 00014 希慎興業s 
Getting 00015 盈信控股 
Getting 00016 新鴻基地產s 
Getting 00017 新世界發展s 
Getting 00018 東方報業集團 
Getting 00019 太古股份公司Ａs 
Getting 00020 會德豐s 
Getting 00021 大中華地產控股 
Getting 00022 茂盛控股 
Getting 00023 東亞銀行s 
Getting 00024 寶威控股 
Getting 00025 其士國際集團s 
Getting 00026 中華汽車s 
Getting 00027 銀河娛樂s 
Getting 00028 天安s 
Getting 00029 達力集團s 
Getting 00030 萬隆控股集團 
Getting 00031 航天控股 
Getting 00032 港通控股s 
Getting 00033 雲銅股份 
Getting 00034 九龍建業s 
Getting 00035 遠東發展s 
Getting 00036 遠東控股國際 
Getting 00037 遠東酒店實業 
Getting 00038 第一拖拉機股份 
Getting 00039 中國北大荒 
Getting 00040 金山工業 
Getting 00041 鷹君s 
Getting 00042 東北電氣 
Getting 00043 卜蜂國際s 
Getting 00045 大酒店s 
Getting 00046 科聯系統 
Getting 00047 合興集團 
Getting 00048 中國汽車內飾 
Getting 00050 香港小輪（集團）

Getting 00419 華誼騰訊娛樂 
Getting 00420 福田實業 
Getting 00422 越南製造加工出口 
Getting 00423 經濟日報集團 
Getting 00425 敏實集團s 
Getting 00426 萬華媒體 
Getting 00428 中國天弓控股 
Getting 00430 東方網庫 
Getting 00431 大中華金融 
Getting 00432 盈大地產 
Getting 00433 北方礦業 
Getting 00434 博雅互動 
Getting 00435 陽光房地產基金s 
Getting 00436 新宇環保 
Getting 00438 彩虹新能源 
Getting 00439 光啟科學 
Getting 00440 大新金融s 
Getting 00442 海福德集團 
Getting 00444 Sincere Watch HK 
Getting 00445 中集天達s 
Getting 00449 志高控股 
Getting 00450 鴻興印刷集團 
Getting 00451 協鑫新能源s 
Getting 00455 天大藥業 
Getting 00456 新城市建設發展 
Getting 00458 聯亞集團 
Getting 00459 美聯工商舖 
Getting 00460 四環醫藥s 
Getting 00462 天然乳品 
Getting 00464 中國海外諾信 
Getting 00465 富通科技 
Getting 00467 聯合能源集團s 
Getting 00468 紛美包裝s 
Getting 00469 凱普松國際 
Getting 00471 中播控股 
Getting 00472 新絲路文旅 
Getting 00474 昊天發展集團 
Getting 00475 中發展控股 
Getting 00476 中國動力控股 
Getting 00479 華建控股 
Getting 00480 香港興業國際s 
Getting 00482 聖馬丁國際 
Getting 00483 包浩斯國際 
Getting 00484 雲遊控股 
Getting 00485 中國華星 
Getting 00486 俄鋁s 
Getting 00487 實德環球 
Ge

Getting 00896 興勝創建 
Getting 00897 位元堂 
Getting 00898 萬事昌國際 
Getting 00899 亞洲資源 
Getting 00900 Aeon信貸財務 
Getting 00901 鷹力投資 
Getting 00902 華能國際電力股份s 
Getting 00904 中綠 
Getting 00905 環球大通投資 
Getting 00906 中糧包裝s 
Getting 00907 高雅光學 
Getting 00908 珠海控股投資 
Getting 00910 中國三迪 
Getting 00911 前海健康 
Getting 00912 信佳國際 
Getting 00913 合一投資 
Getting 00914 海螺水泥s 
Getting 00915 道和環球 
Getting 00916 龍源電力s 
Getting 00918 國能國際資產 
Getting 00919 現代美容 
Getting 00921 海信家電集團s 
Getting 00922 安賢園中國 
Getting 00923 綜合環保集團 
Getting 00924 坤集團 
Getting 00925 北京建設 
Getting 00926 碧生源控股 
Getting 00927 富士高實業 
Getting 00928 蓮和醫療 
Getting 00929 國際精密 
Getting 00931 中國天然氣 
Getting 00932 順騰國際控股 
Getting 00933 光匯石油 
Getting 00934 中石化冠德s 
Getting 00935 龍翔集團 
Getting 00936 鵬程亞洲 
Getting 00938 民生國際 
Getting 00939 建設銀行s 
Getting 00941 中國移動s 
Getting 00943 意科控股 
Getting 00945 宏利金融 
Getting 00947 摩比發展 
Getting 00948 阿爾法企業 
Getting 00950 李氏大藥廠s 
Getting 00951 超威動力s 
Getting 00952 中國通海金融 
Getting 00953 邵氏兄弟控股 
Getting 00954 常茂生物化學工程

Getting 01381 粵豐環保電力s 
Getting 01382 互太紡織s 
Getting 01383 太陽城集團s 
Getting 01385 上海復旦s 
Getting 01386 國投集團控股 
Getting 01387 中國地利s 
Getting 01388 安莉芳控股 
Getting 01389 美捷匯控股 
Getting 01393 恒鼎實業 
Getting 01395 強泰環保 
Getting 01396 毅德國際控股 
Getting 01397 碧瑤綠色集團 
Getting 01398 工商銀行s 
Getting 01399 飛毛腿 
Getting 01400 滿地科技股份 
Getting 01401 Sprocomm Intel 
Getting 01402 超智能控股 
Getting 01410 安領國際控股 
Getting 01412 雋思集團 
Getting 01415 高偉電子 
Getting 01416 CTR Holdings 
Getting 01417 浦江中國 
Getting 01418 盛諾集團 
Getting 01419 盈健醫療 
Getting 01420 川控股 
Getting 01421 工蓋有限公司 
Getting 01425 捷隆控股 
Getting 01426 春泉產業信託s 
Getting 01427 中國天保集團 
Getting 01428 耀才證券金融 
Getting 01430 蘇創燃氣股份 
Getting 01431 原生態牧業 
Getting 01432 中國聖牧奶業 
Getting 01433 常達控股 
Getting 01439 移動互聯（中國） 
Getting 01442 Infinity L&T 
Getting 01443 富臨集團控股 
Getting 01446 鴻福堂集團 
Getting 01447 新福港 
Getting 01448 福壽園國際s 
Getting 01450 世紀睿科 
Getting 01451 萬成集團股份 
Getting 01452 迪諾斯環保 
Getting 01456 國聯證券 
Getting 01458 周黑鴨國際s 
Getting 01459 巨匠建設 
Getting

Getting 01909 火岩控股 
Getting 01910 新秀麗s 
Getting 01911 華興資本控股s 
Getting 01912 康特隆科技 
Getting 01913 普拉達s 
Getting 01915 泰和小貸 
Getting 01916 江西銀行s 
Getting 01917 豆盟科技 
Getting 01918 融創中國s 
Getting 01919 中遠海控s 
Getting 01920 恆新豐控股 
Getting 01921 達力普控股 
Getting 01922 銀城生活服務 
Getting 01925 曠世控股 
Getting 01928 金沙中國有限公司s 
Getting 01929 周大福s 
Getting 01930 勛龍 
Getting 01931 華檢醫療 
Getting 01932 中漆集團 
Getting 01933 元力控股 
Getting 01935 嘉宏教育 
Getting 01937 佳辰控股 
Getting 01938 珠江鋼管 
Getting 01939 東京中央拍賣 
Getting 01941 燁星集團 
Getting 01942 MOG Holdings 
Getting 01943 銀濤控股 
Getting 01949 Platt Nera 
Getting 01950 深藍科技控股 
Getting 01951 錦欣生殖s 
Getting 01955 莊臣控股 
Getting 01958 北京汽車股份s 
Getting 01959 世紀聯合控股 
Getting 01960 TBK & Sons Hldgs 
Getting 01961 九尊數字互娛 
Getting 01962 訓修實業集團 
Getting 01963 重慶銀行股份s 
Getting 01966 中駿集團控股s 
Getting 01967 信懇智能 
Getting 01968 興紡控股 
Getting 01969 中國春來 
Getting 01970 IMAX Chinas 
Getting 01972 太古地產s 
Getting 01975 新興印刷 
Getting 01977 安樂工程 
Getting 01978 敘福樓集團 
Getting 019

Getting 03188 華夏滬深三百s 
Getting 03199 南方政銀國債s 
Getting 03300 中國玻璃 
Getting 03301 融信中國s 
Getting 03302 光控精技 
Getting 03303 巨濤海洋石油服務 
Getting 03306 江南布衣s 
Getting 03308 金鷹商貿集團s 
Getting 03309 希瑪眼科s 
Getting 03311 中國建築國際s 
Getting 03313 雅高控股 
Getting 03315 金邦達寶嘉 
Getting 03316 濱江服務 
Getting 03318 中國香精香料 
Getting 03319 雅生活服務s 
Getting 03320 華潤醫藥s 
Getting 03321 偉鴻集團控股 
Getting 03322 永嘉集團 
Getting 03323 中國建材s 
Getting 03326 保發集團 
Getting 03328 交通銀行s 
Getting 03329 交銀國際 
Getting 03330 靈寶黃金 
Getting 03331 維達國際s 
Getting 03332 南京中生聯合 
Getting 03333 中國恒大s 
Getting 03335 DBA電訊 
Getting 03336 巨騰國際s 
Getting 03337 安東油田服務s 
Getting 03339 中國龍工s 
Getting 03344 共享集團 
Getting 03348 中國鵬飛集團 
Getting 03358 榮威國際控股 
Getting 03360 遠東宏信s 
Getting 03363 正業國際 
Getting 03366 華僑城（亞洲） 
Getting 03368 百盛集團 
Getting 03369 秦港股份 
Getting 03377 遠洋集團s 
Getting 03378 廈門港務 
Getting 03380 龍光地產控股s 
Getting 03382 天津港發展s 
Getting 03383 雅居樂集團s 
Getting 03389 亨得利 
Getting 03390 滿貫集團 
Getting 03393 威勝控股s 
Getting 03395 Persta Reso

Getting 08153 科地農業 
Getting 08155 南華資產控股 
Getting 08156 國藥科技股份 
Getting 08158 中國再生醫學 
Getting 08159 輝煌科技 
Getting 08160 金匯教育 
Getting 08161 醫匯集團 
Getting 08162 港銀控股 
Getting 08163 領智金融 
Getting 08165 華普智通 
Getting 08166 中國農業生態 
Getting 08167 中國新電信 
Getting 08168 寶積資本 
Getting 08169 環康集團 
Getting 08170 全民國際 
Getting 08171 中國趨勢 
Getting 08172 拉近網娛 
Getting 08173 萬亞企業控股 
Getting 08175 中國數碼文化 
Getting 08176 超人智能 
Getting 08178 中國信息科技 
Getting 08179 新煮意控股 
Getting 08181 時時服務 
Getting 08186 同仁資源 
Getting 08187 積木集團 
Getting 08188 駿傑集團控股 
Getting 08189 泰達生物 
Getting 08190 康佰控股 
Getting 08191 鴻偉亞洲 
Getting 08192 環球通証 
Getting 08193 亞太金融投資 
Getting 08195 樂亞國際控股 
Getting 08196 建禹集團 
Getting 08198 樂透互娛 
Getting 08200 修身堂 
Getting 08201 寶聯控股 
Getting 08202 匯創控股 
Getting 08203 凱順控股 
Getting 08205 交大慧谷 
Getting 08206 神通機器人教育 
Getting 08207 中新控股 
Getting 08208 WMCH Global 
Getting 08210 衍匯亞洲 
Getting 08211 浙江永安 
Getting 08212 譽滿國際控股 
Getting 08213 榮暉控股 
Getting 08215 第一信用金融 
Getting 08216 豐城控股 
Getti

In [32]:
allStockDf

,code,name,截止日期,盈利(百萬),盈利增長(%),每股盈利,每股盈利增長(%),每股派息,市盈率(倍),周息率(%),派息比率(%),每股賬面資產淨值,基準貨幣,兌換比率
0,00001,長江和記實業s,2015/12,118570,120.11,36.9100,58.68,2.5500,2.61,2.64,6.91,101.9347,港元,1
1,00001,長江和記實業s,2016/12,33008,-72.16,8.5500,-76.84,2.6800,11.57,2.71,31.35,102.1778,港元,1
2,00001,長江和記實業s,2017/12,35100,6.34,9.1000,6.43,2.8500,8.97,3.49,31.32,111.4805,港元,1
3,00001,長江和記實業s,2018/12,39000,11.11,10.1100,11.10,3.1700,4.51,6.96,31.36,115.6958,港元,1
4,00001,長江和記實業s,2019/12,39830,2.13,10.3300,2.18,3.1700,5.35,5.74,30.69,120.3984,港元,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13145,87001,匯賢產業信託s,2015/12,168,-88.84,0.0315,-89.05,0.2700,100.32,8.54,857.14,5.1231,人民幣,1
13146,87001,匯賢產業信託s,2016/12,114,-32.14,0.0211,-33.02,0.2754,154.98,8.42,1.31K,4.8404,人民幣,1
13147,87001,匯賢產業信託s,2017/12,1456,1.18K,0.2635,1.15K,0.2681,12.87,7.91,101.75,4.7943,人民幣,1
13148,87001,匯賢產業信託s,2018/12,574,-60.58,0.1012,-61.59,0.2653,27.67,9.48,262.15,4.5808,人民幣,1


In [33]:
# allStockDf.drop_duplicates(keep=False, inplace=True)
# allStockDf.to_csv('./all_origin.csv', encoding="utf_8_sig" , index=False)
# allStockDf

,code,name,截止日期,盈利(百萬),盈利增長(%),每股盈利,每股盈利增長(%),每股派息,市盈率(倍),周息率(%),派息比率(%),每股賬面資產淨值,基準貨幣,兌換比率
0,00001,長江和記實業s,2015/12,118570,120.11,36.9100,58.68,2.5500,2.61,2.64,6.91,101.9347,港元,1
1,00001,長江和記實業s,2016/12,33008,-72.16,8.5500,-76.84,2.6800,11.57,2.71,31.35,102.1778,港元,1
2,00001,長江和記實業s,2017/12,35100,6.34,9.1000,6.43,2.8500,8.97,3.49,31.32,111.4805,港元,1
3,00001,長江和記實業s,2018/12,39000,11.11,10.1100,11.10,3.1700,4.51,6.96,31.36,115.6958,港元,1
4,00001,長江和記實業s,2019/12,39830,2.13,10.3300,2.18,3.1700,5.35,5.74,30.69,120.3984,港元,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13145,87001,匯賢產業信託s,2015/12,168,-88.84,0.0315,-89.05,0.2700,100.32,8.54,857.14,5.1231,人民幣,1
13146,87001,匯賢產業信託s,2016/12,114,-32.14,0.0211,-33.02,0.2754,154.98,8.42,1.31K,4.8404,人民幣,1
13147,87001,匯賢產業信託s,2017/12,1456,1.18K,0.2635,1.15K,0.2681,12.87,7.91,101.75,4.7943,人民幣,1
13148,87001,匯賢產業信託s,2018/12,574,-60.58,0.1012,-61.59,0.2653,27.67,9.48,262.15,4.5808,人民幣,1


In [34]:
allStockDf = pd.read_csv('./all_origin.csv')
allStockDf = allStockDf.loc[:, ~allStockDf.columns.str.contains('^Unnamed')]
display(allStockDf.dtypes)
display(allStockDf.head())

code           int64
name          object
截止日期          object
盈利(百萬)        object
盈利增長(%)       object
每股盈利          object
每股盈利增長(%)     object
每股派息          object
市盈率(倍)        object
周息率(%)        object
派息比率(%)       object
每股賬面資產淨值      object
基準貨幣          object
兌換比率         float64
dtype: object

,code,name,截止日期,盈利(百萬),盈利增長(%),每股盈利,每股盈利增長(%),每股派息,市盈率(倍),周息率(%),派息比率(%),每股賬面資產淨值,基準貨幣,兌換比率
0,1,長江和記實業s,2015/12,118570,120.11,36.9100,58.68,2.5500,2.61,2.64,6.91,101.9347,港元,1.0
1,1,長江和記實業s,2016/12,33008,-72.16,8.5500,-76.84,2.6800,11.57,2.71,31.35,102.1778,港元,1.0
2,1,長江和記實業s,2017/12,35100,6.34,9.1000,6.43,2.8500,8.97,3.49,31.32,111.4805,港元,1.0
3,1,長江和記實業s,2018/12,39000,11.11,10.1100,11.10,3.1700,4.51,6.96,31.36,115.6958,港元,1.0
4,1,長江和記實業s,2019/12,39830,2.13,10.3300,2.18,3.1700,5.35,5.74,30.69,120.3984,港元,1.0


In [35]:
allStockDf.columns = ['code', 'name', 'time' , 
                      'netProfit ' , 'netProfit Growth' , 'eps',
                      'epsGrowth' , 'interestRates' , 'PE' , 
                      'dividendYield' ,'payoutRatio' , 'netAssetValue' , 
                      'currency' , 'rate' ]

In [36]:
allStockDf.sample(10)

,code,name,time,netProfit,netProfit Growth,eps,epsGrowth,interestRates,PE,dividendYield,payoutRatio,netAssetValue,currency,rate
708,150,Hypebeast,2018/03,45,93.83,0.0226,93.16,0.0000,0.00,-,0.00,0.0706,港元,1.0000
8469,2266,黎氏企業,2017/12,18,-40.19,0.0447,-54.46,0.0000,15.00,0.00,0.00,0.4738,港元,0.9709
7250,1803,北京體育文化,2019/12,-63,-14.73,-0.0490,-16.67,0.0000,-4.57,0.00,0.00,0.2601,港元,1.0000
1228,261,大灣區投資控股s,2018/12,-88,55.56,-0.0006,60.00,0.0000,-16.67,0.00,0.00,0.0064,港元,1.0000
10174,3958,東方證券s,2018/12,1402,-67.10,0.2049,-70.01,0.1139,19.86,2.80,55.56,8.4234,港元,1.1386
9211,2863,高豐集團控股,2017/09,22,-26.36,0.0500,-82.76,0.0000,0.00,-,0.00,0.2951,港元,1.0000
10002,3813,寶勝國際s,2019/12,931,50.64,0.1773,49.91,0.0000,8.46,0.00,0.00,-,港元,1.1175
161,34,九龍建業s,2016/12,947,-21.24,0.8200,-21.15,0.6000,10.85,6.74,73.17,20.4919,港元,1.0000
3017,693,陳唱國際,2019/12,213,-64.56,0.1100,-63.33,0.1100,16.82,5.95,100.00,-,港元,1.0000
10506,6190,九江銀行,2019/12,2053,2.59,0.8493,-6.76,0.1341,0.00,-,15.79,11.4776,港元,1.1175


In [37]:
allStockDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12864 entries, 0 to 12863
Data columns (total 14 columns):
code                12864 non-null int64
name                12864 non-null object
time                12637 non-null object
netProfit           12637 non-null object
netProfit Growth    12637 non-null object
eps                 12637 non-null object
epsGrowth           12637 non-null object
interestRates       12637 non-null object
PE                  12637 non-null object
dividendYield       12637 non-null object
payoutRatio         12637 non-null object
netAssetValue       12637 non-null object
currency            12637 non-null object
rate                12637 non-null float64
dtypes: float64(1), int64(1), object(12)
memory usage: 1.4+ MB


In [38]:
round(allStockDf.isnull().sum()/len(allStockDf)*100, 2)

code                0.00
name                0.00
time                1.76
netProfit           1.76
netProfit Growth    1.76
eps                 1.76
epsGrowth           1.76
interestRates       1.76
PE                  1.76
dividendYield       1.76
payoutRatio         1.76
netAssetValue       1.76
currency            1.76
rate                1.76
dtype: float64

In [40]:
allStockDf = allStockDf.replace("-", None)

In [41]:
allStockDf[allStockDf.apply(lambda row: 
                            row.astype(str).str.endswith('K').any() or 
                            row.astype(str).str.endswith('M').any() , 
                            axis=1)]

,code,name,time,netProfit,netProfit Growth,eps,epsGrowth,interestRates,PE,dividendYield,payoutRatio,netAssetValue,currency,rate
33,7,香港金融集團,2018/12,160,1.27K,0.0455,889.13,0.0000,17.58,0.00,0.00,0.7667,港元,1.0000
187,39,中國北大荒,2017/12,115,1.49K,0.0220,1.37K,0.0000,12.50,0.00,0.00,0.2691,港元,1.0000
445,95,綠景中國地產s,2015/12,491,2.91K,0.1241,925.44,0.0100,17.41,0.46,8.06,1.9748,港元,1.1761
453,96,友成控股,2018/12,104,3.99,0.0017,-99.16,0.0182,510.89,2.05,1.05K,1.4228,港元,1.1386
454,96,友成控股,2019/12,40,-61.81,0.0006,-64.72,0.0145,1.37K,1.73,2.36K,1.4228,港元,1.1175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12630,8668,瀛海集團,2017/12,18,-17.64,0.0000,-33.63,0.0000,0.00,NaN,0.00,537.8600K,港元,1.0000
12631,8668,瀛海集團,2018/12,16,-10.66,0.0000,-33.63,0.0000,0.00,NaN,0.00,537.8600K,港元,1.0000
12685,9918,麗年國際,2017/12,74,7.45K,0.0000,NaN,0.0000,NaN,0.00,NaN,NaN,港元,1.0000
12860,87001,匯賢產業信託s,2016/12,114,-32.14,0.0211,-33.02,0.2754,154.98,8.42,1.31K,4.8404,人民幣,1.0000


In [43]:
allStockDf.drop(
    allStockDf.index[
        allStockDf.apply(lambda row: row.astype(str).str.endswith('K').any() or 
                         row.astype(str).str.endswith('M').any() , 
                         axis=1)] 
    , inplace = True )

In [44]:
allStockDf[allStockDf.apply(lambda row: 
                            row.astype(str).str.endswith('K').any() or 
                            row.astype(str).str.endswith('M').any() , 
                            axis=1)]
display(allStockDf.shape)

(12628, 14)

In [45]:
allStockDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12628 entries, 0 to 12863
Data columns (total 14 columns):
code                12628 non-null int64
name                12628 non-null object
time                12401 non-null object
netProfit           12401 non-null object
netProfit Growth    11963 non-null object
eps                 12401 non-null object
epsGrowth           11777 non-null object
interestRates       12401 non-null object
PE                  11958 non-null object
dividendYield       10765 non-null object
payoutRatio         11958 non-null object
netAssetValue       11700 non-null object
currency            12401 non-null object
rate                12401 non-null float64
dtypes: float64(1), int64(1), object(12)
memory usage: 1.4+ MB


In [47]:
#類型變更
allStockDf[allStockDf.columns.difference(['code', 'name' , 'currency' , 'time' ])] = allStockDf[allStockDf.columns.difference(['code', 'name' , 'currency' , 'time' ])].astype(float)
#Convert Quarter end to datetime
allStockDf['time'] = pd.to_datetime(allStockDf['time'])
#Generate a Year and Month column from Quarter end
allStockDf['year'] = allStockDf['time'].dt.year.astype('Int64')
allStockDf['month'] = allStockDf['time'].dt.month.astype('Int64')
allStockDf['code'] = allStockDf['code'].astype(str)
allStockDf['code'] = allStockDf['code'].str.zfill(5)
allStockDf


,code,name,time,netProfit,netProfit Growth,eps,epsGrowth,interestRates,PE,dividendYield,payoutRatio,netAssetValue,currency,rate,year,month
0,00001,長江和記實業s,2015-12-01,118570.0,120.11,36.9100,58.68,2.5500,2.61,2.64,6.91,101.9347,港元,1.0,2015,12
1,00001,長江和記實業s,2016-12-01,33008.0,-72.16,8.5500,-76.84,2.6800,11.57,2.71,31.35,102.1778,港元,1.0,2016,12
2,00001,長江和記實業s,2017-12-01,35100.0,6.34,9.1000,6.43,2.8500,8.97,3.49,31.32,111.4805,港元,1.0,2017,12
3,00001,長江和記實業s,2018-12-01,39000.0,11.11,10.1100,11.10,3.1700,4.51,6.96,31.36,115.6958,港元,1.0,2018,12
4,00001,長江和記實業s,2019-12-01,39830.0,2.13,10.3300,2.18,3.1700,5.35,5.74,30.69,120.3984,港元,1.0,2019,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12857,83199,南方政銀國債－Ｒs,2017-12-01,0.0,NaN,0.0000,NaN,3.6000,NaN,0.85,NaN,NaN,人民幣,1.0,2017,12
12858,83199,南方政銀國債－Ｒs,2018-12-01,0.0,NaN,0.0000,NaN,3.6000,NaN,0.85,NaN,NaN,人民幣,1.0,2018,12
12859,87001,匯賢產業信託s,2015-12-01,168.0,-88.84,0.0315,-89.05,0.2700,100.32,8.54,857.14,5.1231,人民幣,1.0,2015,12
12862,87001,匯賢產業信託s,2018-12-01,574.0,-60.58,0.1012,-61.59,0.2653,27.67,9.48,262.15,4.5808,人民幣,1.0,2018,12


In [49]:
allStockDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12628 entries, 0 to 12863
Data columns (total 16 columns):
code                12628 non-null object
name                12628 non-null object
time                12401 non-null datetime64[ns]
netProfit           12401 non-null float64
netProfit Growth    11963 non-null float64
eps                 12401 non-null float64
epsGrowth           11777 non-null float64
interestRates       12401 non-null float64
PE                  11958 non-null float64
dividendYield       10765 non-null float64
payoutRatio         11958 non-null float64
netAssetValue       11700 non-null float64
currency            12401 non-null object
rate                12401 non-null float64
year                12401 non-null Int64
month               12401 non-null Int64
dtypes: Int64(2), datetime64[ns](1), float64(10), object(3)
memory usage: 1.7+ MB


In [53]:
allStockDf.to_csv('./all_cleaned.csv', encoding="utf_8_sig" , index=False)

In [54]:
stockListDf.to_csv('./hk_stock_list.csv', encoding="utf_8_sig" , index=False)

In [51]:
allStockDf

,code,name,time,netProfit,netProfit Growth,eps,epsGrowth,interestRates,PE,dividendYield,payoutRatio,netAssetValue,currency,rate,year,month
0,00001,長江和記實業s,2015-12-01,118570.0,120.11,36.9100,58.68,2.5500,2.61,2.64,6.91,101.9347,港元,1.0,2015,12
1,00001,長江和記實業s,2016-12-01,33008.0,-72.16,8.5500,-76.84,2.6800,11.57,2.71,31.35,102.1778,港元,1.0,2016,12
2,00001,長江和記實業s,2017-12-01,35100.0,6.34,9.1000,6.43,2.8500,8.97,3.49,31.32,111.4805,港元,1.0,2017,12
3,00001,長江和記實業s,2018-12-01,39000.0,11.11,10.1100,11.10,3.1700,4.51,6.96,31.36,115.6958,港元,1.0,2018,12
4,00001,長江和記實業s,2019-12-01,39830.0,2.13,10.3300,2.18,3.1700,5.35,5.74,30.69,120.3984,港元,1.0,2019,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12857,83199,南方政銀國債－Ｒs,2017-12-01,0.0,NaN,0.0000,NaN,3.6000,NaN,0.85,NaN,NaN,人民幣,1.0,2017,12
12858,83199,南方政銀國債－Ｒs,2018-12-01,0.0,NaN,0.0000,NaN,3.6000,NaN,0.85,NaN,NaN,人民幣,1.0,2018,12
12859,87001,匯賢產業信託s,2015-12-01,168.0,-88.84,0.0315,-89.05,0.2700,100.32,8.54,857.14,5.1231,人民幣,1.0,2015,12
12862,87001,匯賢產業信託s,2018-12-01,574.0,-60.58,0.1012,-61.59,0.2653,27.67,9.48,262.15,4.5808,人民幣,1.0,2018,12


In [55]:
allStockDf.dtypes

code                        object
name                        object
time                datetime64[ns]
netProfit                  float64
netProfit Growth           float64
eps                        float64
epsGrowth                  float64
interestRates              float64
PE                         float64
dividendYield              float64
payoutRatio                float64
netAssetValue              float64
currency                    object
rate                       float64
year                         Int64
month                        Int64
dtype: object

In [63]:
allStockDf = allStockDf['code'].str.slice(2,7)
allStockDf

0        00001
1        00001
2        00001
3        00001
4        00001
         ...  
12857    83199
12858    83199
12859    87001
12862    87001
12863    87001
Name: code, Length: 12628, dtype: object